In [17]:
import json
import requests
import praw

reddit = praw.Reddit(client_id='tiBit4kBeuoOTA', 
                     client_secret='7-HPDa9q92MS0a-AnUVYb72QDzU', 
                     user_agent='TorkilSamStat', 
                     username='TorkSamStat', 
                     password='bbybbyduerinnafor')

# STATIC VARIABLES
base_url = 'https://api.pushshift.io/reddit/search/comment/'
wanted_count_auth = 2
wanted_count_sub = 2
unwanted_users = ["[deleted]", "AutoModerator"]
unwanted_count = len(unwanted_users)
used_authors = []
used_subreddits = ['the_donald']
 
# FUNCTIONS
def getKeysFromDict(data):
    users = []
    for obj in data:
        if obj['key'] not in ['AutoModerator', '[deleted]']:
            users.append(obj['key'])
    return users

def getMostActiveAuthorsInSubReddit(subreddit):
    authors = []
    offset = 0
    while (len(authors) < wanted_count_auth):
        url = base_url + '?since=100d&subreddit={}&aggs=author&agg_size={}'.format(subreddit, wanted_count_auth + unwanted_count + offset)
        response = requests.get(url).json()
        fetchedAuthors = getKeysFromDict(response['aggs']['author'])
        for auth in fetchedAuthors:
            if (auth not in used_authors):
                authors.append(auth)
                used_authors.append(auth)
        offset += 1
    return authors

def getMostActiveSubRedditForAuthor(author):
    subreddits = []
    offset = 0
    while (len(subreddits) < wanted_count_sub):
        url = base_url + '?since=100d&author={}&aggs=subreddit&agg_size={}'.format(author, wanted_count_sub + offset)
        response = requests.get(url).json()
        fetchedSubreddits = getKeysFromDict(response['aggs']['subreddit'])
        for sub in fetchedSubreddits:
            if (sub not in used_subreddits):
                subreddits.append(sub)
                used_subreddits.append(sub)
        offset += 1
    return subreddits

In [18]:
# Get users of level 1
users_1 = getMostActiveAuthorsInSubReddit('the_donald')
users_1

['mangoroti', 'ENVYNITAZ']

In [19]:
subs_1 = getMostActiveSubRedditForAuthor('mangoroti')
subs_1

['The_Donald', 'florida']